In [8]:
import requests
from bs4 import BeautifulSoup

def crawler_func(seed, maxcount):
    Link_person = [seed]
    count = 0
    while(Link_person!=[] and count < maxcount):
        count +=1
        url = Link_person[0]
        Link_person = Link_person[1:]
        code = requests.get(url)
        plain = code.text
        B_S = BeautifulSoup(plain, "html.parser")
        for link in B_S.findAll('a'):
            new_url = link.get('href')
            if(new_url != None and new_url != '/'):
                new_url = new_url.strip()
                if( new_url[0:7] != 'http://' and new_url[0:8]!='https://' ) :
                    if(url[len(url) -1] == '/'):
                        if(new_url[0] == '/'):
                            url_split = new_url.split('/')
                            for k in range(len(url_split)):
                                if(url_split[k] == 'persons'):
                                    new_url=seed+'/'+url_split[k]+'/'
                                    return new_url
                            new_url = url + new_url[1:len(new_url)]
                    else:
                        new_url = url + '/' + new_url
            Link_person.append(new_url)         
all_persons=crawler_func('https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences',50)
print(all_persons)

https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences/persons/


In [9]:
page_num = 0
all_publication_pages = all_persons+ "/?page="
author_url_initial=[]
author_url=[]

while True:
    all_publication_pages = all_publication_pages + str(page_num)
    auth_page_url = requests.get(all_publication_pages) 
    bs_auth_url=BeautifulSoup(auth_page_url.content, 'html.parser')
    author_url=bs_auth_url.find_all('div',class_='rendering rendering_person rendering_short rendering_person_short')
    if author_url :
        for j in author_url:
            author_url_initial.append(j.find(class_='link person').get('href'))
    else :
        break
    page_num= page_num +1

In [10]:
import re
author_url_final=[]
title=[]
link=[]
authors=[]
publication_date=[]
def get_sls_data(url_link,result):
    Type_of_pub = ["Article", "Paper"]
    auth_page = requests.get(url_link)
    bs_auth=BeautifulSoup(auth_page.content, 'html.parser')
    auth_art=bs_auth.find_all('div',class_=re.compile("^rendering rendering_researchoutput rendering_researchoutput_portal-short rendering_contributiontojournal rendering_portal-short.*"))
    if auth_art:
        for j in auth_art:
            try:
                if(j.find(class_="type_classification").text in Type_of_pub):   
                    author_url_final.append(url_link)
                    title.append(j.h3.a.text)
                    link.append(j.h3.a.get('href'))
                    text_p=j.text
                    authors.append(text_p.replace(j.h3.a.text,""))
                    publication_date.append(j.find(class_="date").text)
            except:
                pass
        return result
    else:
        result =  False
        return result

In [11]:
for j in range(0,len(author_url_initial)):

    result = True
    page_num = 0
    while result:
        url = author_url_initial[j]+"/publications/?page=" + str(page_num)
        result = get_sls_data(url,result) 
        page_num=page_num +1
        if not result:
            break
            
for index, item in enumerate(authors):
    item = authors[index].split(publication_date[index],)[0]
    authors[index] = item

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
full_sentence=[]

def data_preprocessing():
    
    #lowercase
    title_low =[]
    for i in title:
        title_low.append(i.lower())
    print(title_low)
    
    # punctuation removel
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    title_low_no_punct=[]
    for i in title_low:
        no_punct = ""
        for char in i:
            if char not in punctuations:
                no_punct = no_punct + char
            elif char in punctuations:
                no_punct = no_punct + ' '
        title_low_no_punct.append(no_punct)
    print(title_low_no_punct)
    
    #remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_sentence=[]
    for i in title_low_no_punct:
        word_tokens = word_tokenize(i)
        filtered_sentence = ([w for w in word_tokens if not w.lower() in stop_words])
        full_sentence.append((" ").join(filtered_sentence))
    print(full_sentence)
    
    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
data_preprocessing()

['examining the presence of cronobacter spp. in ready-to-eat edible insects', 'investigating the transmissibility of tet(w) in bifidobacteria exposed to acid and bile stress', 'sugar and cocoa: sweet synergy or bitter antagonisms: formulating cocoa and chocolate products for health: a narrative review', 'exploring the relationship between exposure to technological and gastrointestinal stress and probiotic functional properties of lactobacilli and bifidobacteria', 'assessing the effects of exposure to environmental stress on some functional properties of bifidobacterium animalis ssp. lactis', 'a communal language for decision making in team invasion sports', 'what cognitive mechanism, when, where, and why? exploring the decision making of university and professional rugby union players during competitive matches', 'the impact of rule modifications on player behaviour in a talent identification and development environment: a case study of the rugby football union’s wellington academy rug

In [14]:
#inverted index
inverted_index = {}

for i, doc in enumerate(full_sentence):
    for term in doc.split():
        if term in inverted_index:
            inverted_index[term].add(i)
        else: inverted_index[term] = {i}

print(inverted_index)

{'examining': {0, 226, 241, 91, 479}, 'presence': {0, 444, 424}, 'cronobacter': {0, 491}, 'spp': {0, 492}, 'ready': {0}, 'eat': {0}, 'edible': {0}, 'insects': {0}, 'investigating': {1}, 'transmissibility': {1}, 'tet': {1}, 'w': {1}, 'bifidobacteria': {1, 3}, 'exposed': {1}, 'acid': {488, 1, 268, 478}, 'bile': {1}, 'stress': {1, 3, 4, 454, 455, 318}, 'sugar': {2}, 'cocoa': {2}, 'sweet': {2}, 'synergy': {2, 366}, 'bitter': {2}, 'antagonisms': {2}, 'formulating': {2}, 'chocolate': {2}, 'products': {2}, 'health': {161, 2, 165, 458, 374, 279, 350, 375}, 'narrative': {2}, 'review': {2, 431, 21, 22, 350, 351}, 'exploring': {196, 3, 190, 6}, 'relationship': {384, 3, 198, 410, 59, 413}, 'exposure': {3, 4}, 'technological': {3}, 'gastrointestinal': {3}, 'probiotic': {3, 174, 178, 179, 180}, 'functional': {257, 3, 4, 103, 264, 107, 75, 305, 313, 348}, 'properties': {466, 3, 4}, 'lactobacilli': {3}, 'assessing': {264, 4, 205, 462}, 'effects': {256, 385, 4, 263, 137, 281, 155, 411, 285, 291, 292, 2

In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import collections

def output_Func(user_input):
    input_low =[]
    input_low.append(user_input.lower())
    print(input_low)
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    input_low_no_punct=[]
    for i in input_low:
        no_punct = ""
        for char in i:
            if char not in punctuations:
                no_punct = no_punct + char
            elif char in punctuations:
                if char == '-':
                    no_punct = no_punct + ' '
                else:
                    no_punct = no_punct + ''
        
        input_low_no_punct.append(no_punct)
    print(input_low_no_punct)
    stop_words = set(stopwords.words('english'))
    final_filtered_sentence=[]
    for i in input_low_no_punct:
        word_tokens = word_tokenize(i)
        final_filtered_sentence.append([w for w in word_tokens if not w.lower() in stop_words])
    
    print(final_filtered_sentence)
    
    inv_ind_list = []
    for i in range(len(final_filtered_sentence[0])):
        try:
          inv_ind_list.append(inverted_index[final_filtered_sentence[0][i]])
          final_inv_ind_list = []
        except:
          print(final_filtered_sentence[0], "is not found.")   
    for i in range(len(inv_ind_list)):
        inv_ind_list[i] = list(inv_ind_list[i])
        final_inv_ind_list = final_inv_ind_list + inv_ind_list[i]
    Dup_List = []
    Dup_List = [item for item, count in collections.Counter(final_inv_ind_list).items() if count > 1]
    print(Dup_List)
    print("---------------------")
    unq_list = []

    for x in final_inv_ind_list:
        if x not in unq_list:
            unq_list.append(x)
        
    for element in unq_list:
        if element in Dup_List:
            unq_list.remove(element)

    print(unq_list)
    for i in range(len(Dup_List)):
        doc_id = int(Dup_List[i])
        print("Titile is: ",title[doc_id])
        print("Publication Link is: ",link[doc_id])
        print("Author Link is: ",author_url_final[doc_id])
        print("Authors is: ",authors[doc_id])
        print("Publication Date is: ",publication_date[doc_id])
        print("------------------------------------------------------------------------------------")
    for i in range(len(unq_list)):
        doc_id = int(unq_list[i])
        print("Titile is: ",title[doc_id])
        print("Publication Link is: ",link[doc_id])
        print("Author Link is: ",author_url_final[doc_id])
        print("Authors is: ",authors[doc_id])
        print("Publication Date is: ",publication_date[doc_id])
        print("------------------------------------------------------------------------------------")

In [16]:
user_input = input()

output_Func(user_input)

protein & Engineering
['protein & engineering']
['protein  engineering']
[['protein', 'engineering']]
[182, 480]
---------------------
[259, 401, 402, 403, 404, 406, 24, 289, 292, 38, 308, 439, 440, 56, 323, 327, 481, 485]
Titile is:  Protein engineering of Pseudomonas fluorescens peroxidase Dyp1B for oxidation of phenolic and polymeric lignin substrates
Publication Link is:  https://pureportal.coventry.ac.uk/en/publications/protein-engineering-of-pseudomonas-fluorescens-peroxidase-dyp1b-f
Author Link is:  https://pureportal.coventry.ac.uk/en/persons/austine-ehibhatiomhan/publications/?page=0
Authors is:  Pour, R. R., Ehibhatiomhan, A., Huang, Y., Ashley, B., Rashid, G., Mendel Williams, S. & Bugg, T. D. H., 
Publication Date is:  Apr 2019
------------------------------------------------------------------------------------
Titile is:  Protein engineering of Pseudomonas fluorescens peroxidase Dyp1B for oxidation of phenolic and polymeric lignin substrates
Publication Link is:  https://p